In [19]:
import pandas as pd
import networkx as nx
import json

In [14]:
FN = "../Runtimes-duckdb.csv"

In [15]:
df = pd.read_csv(FN)

In [26]:
exp = df["exp_analyze"].values[5]

In [27]:
exp

'[(\'analyzed_plan\', \'{\\n   "name":  "Query", \\n   "result": 0.717119,\\n   "timing": 0.717131,\\n   "cardinality": 0,\\n   "extra-info": "explain analyze SELECT MIN(cn.name) AS from_company,\\\\n       MIN(mc.note) AS production_note,\\\\n       MIN(t.title) AS movie_based_on_book\\\\nFROM company_name AS cn,\\\\n     company_type AS ct,\\\\n     keyword AS k,\\\\n     link_type AS lt,\\\\n     movie_companies AS mc,\\\\n     movie_keyword AS mk,\\\\n     movie_link AS ml,\\\\n     title AS t\\\\nWHERE cn.country_code !=\\\'[pl]\\\'\\\\n  AND (cn.name LIKE \\\'20th Century Fox%\\\'\\\\n       OR cn.name LIKE \\\'Twentieth Century Fox%\\\')\\\\n  AND ct.kind != \\\'production companies\\\'\\\\n  AND ct.kind IS NOT NULL\\\\n  AND k.keyword IN (\\\'sequel\\\',\\\\n                    \\\'revenge\\\',\\\\n                    \\\'based-on-novel\\\')\\\\n  AND mc.note IS NOT NULL\\\\n  AND t.production_year > 1950\\\\n  AND lt.id = ml.link_type_id\\\\n  AND ml.movie_id = t.id\\\\n  AND 

In [35]:
import ast

# Assuming the data string you provided is stored in a variable named 'data'
tuple_data = ast.literal_eval(exp)
json_string = tuple_data[0][1]  # Extracting the second element of the tuple

parsed_data = json.loads(json_string)
graph = nx.DiGraph()


def add_nodes_and_edges(parent, data):
    # Adding the current node to the graph
    graph.add_node(data["name"], **data)
    
    # If this node is not the root node, add an edge from the parent
    if parent:
        graph.add_edge(parent, data["name"])
    
    # If this node has children, call this function recursively for each child
    for child in data.get("children", []):
        add_nodes_and_edges(data["name"], child)

root_data = parsed_data
add_nodes_and_edges(None, root_data)

# print nodes
print(graph.nodes(data=True))

[('Query', {'name': 'Query', 'result': 0.717119, 'timing': 0.717131, 'cardinality': 0, 'extra-info': "explain analyze SELECT MIN(cn.name) AS from_company,\n       MIN(mc.note) AS production_note,\n       MIN(t.title) AS movie_based_on_book\nFROM company_name AS cn,\n     company_type AS ct,\n     keyword AS k,\n     link_type AS lt,\n     movie_companies AS mc,\n     movie_keyword AS mk,\n     movie_link AS ml,\n     title AS t\nWHERE cn.country_code !='[pl]'\n  AND (cn.name LIKE '20th Century Fox%'\n       OR cn.name LIKE 'Twentieth Century Fox%')\n  AND ct.kind != 'production companies'\n  AND ct.kind IS NOT NULL\n  AND k.keyword IN ('sequel',\n                    'revenge',\n                    'based-on-novel')\n  AND mc.note IS NOT NULL\n  AND t.production_year > 1950\n  AND lt.id = ml.link_type_id\n  AND ml.movie_id = t.id\n  AND t.id = mk.movie_id\n  AND mk.keyword_id = k.id\n  AND t.id = mc.movie_id\n  AND mc.company_type_id = ct.id\n  AND mc.company_id = cn.id\n  AND ml.movie_

In [36]:
graph.nodes(data=False)

NodeView(('Query', 'EXPLAIN_ANALYZE', 'UNGROUPED_AGGREGATE', 'PROJECTION', 'HASH_JOIN', 'FILTER', 'SEQ_SCAN '))

In [44]:
parsed_data.keys()

dict_keys(['name', 'result', 'timing', 'cardinality', 'extra-info', 'timings', 'children'])

In [50]:
parsed_data["children"][0]

{'name': 'EXPLAIN_ANALYZE',
 'timing': 0.0,
 'cardinality': 0,
 'extra_info': '',
 'timings': [],
 'children': [{'name': 'UNGROUPED_AGGREGATE',
   'timing': 0.000198,
   'cardinality': 1,
   'extra_info': 'min(#0)\nmin(#1)\nmin(#2)',
   'timings': [],
   'children': [{'name': 'PROJECTION',
     'timing': 7e-06,
     'cardinality': 6946,
     'extra_info': 'name\nnote\ntitle\n',
     'timings': [],
     'children': [{'name': 'HASH_JOIN',
       'timing': 0.000172,
       'cardinality': 6946,
       'extra_info': 'INNER\nlink_type_id = id\n\n[INFOSEPARATOR]\nEC: 19\nCost: 8011',
       'timings': [],
       'children': [{'name': 'HASH_JOIN',
         'timing': 0.229352,
         'cardinality': 6946,
         'extra_info': 'INNER\nid = keyword_id\n\n[INFOSEPARATOR]\nEC: 19\nCost: 7991',
         'timings': [],
         'children': [{'name': 'PROJECTION',
           'timing': 2e-06,
           'cardinality': 3,
           'extra_info': '#1\n',
           'timings': [],
           'children